In [18]:
import pandas as pd
import numpy as np
import sys
import os
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

import utils.data_loader as dl
import database.db_connector as connector

In [89]:
def create_real_weight_column(df: pd.DataFrame,
                      weight_body_exercises: list = None,
                      df_metrics: pd.DataFrame = None) -> pd.DataFrame:
    """
    Merge the metrics dataset with the df to compute the real bodyweight
    in each training and added to the total weight lifted in those bodyweight exercises.
    """
    if weight_body_exercises is None:
        weight_body_exercises = [
        'Chin-ups', 
        'Parallel bar dips',
        'Pull-ups',
        'Muscle-ups',
        'Neutral grip pull-ups',
        'Ring chin-ups',
        'Parallel bar dips 210',
        'Barbell squat'
        ]

    df_merged = df.copy()

    if df_metrics is not None:
        query = """
        SELECT df.*, df_metrics.fecha_prev, df_metrics.peso_prev, df_metrics.delta_daily, df_metrics.fecha AS fecha_next
        FROM df
        LEFT JOIN df_metrics ON df.fecha >= df_metrics.fecha_prev AND df.fecha < df_metrics.fecha
        """

        df_merged = psql.sqldf(query, locals())

        df_merged[['fecha', 'fecha_prev', 'fecha_next']] = df_merged[['fecha', 'fecha_prev', 'fecha_next']].apply(pd.to_datetime)
        
        max_weight = df_metrics.loc[df_metrics.fecha == df_metrics.fecha.max(),'peso']
        max_date = df_metrics.loc[df_metrics.fecha == df_metrics.fecha.max(),'fecha']
        
        df_merged.loc[df_merged.fecha_prev.isnull(),'fecha_prev'] = [max_date]
        df_merged.loc[df_merged.peso_prev.isnull(),'peso_prev'] = [max_weight]

        df_merged['current_bodyweight'] = np.round(
                                            np.where(df_merged.delta_daily.isnull(), df_merged.peso_prev, 
                                                (df_merged.fecha - df_merged.fecha_prev).dt.days * df_merged.delta_daily + df_merged.peso_prev)
                                                    ,2)
        
        df_merged['real_weight'] = np.where(
                            df_merged.exercise.isin(weight_body_exercises), 
                                df_merged.weight + df_merged.current_bodyweight, 
                                    df_merged.weight)
    else:
        df_merged['current_bodyweight'] = np.nan
        df_merged['real_weight'] = df_merged['weight']
    
    return df_merged

def training_days_on_week(df):
    # Training days on week
    if 'training_days_on_week' not in df.columns:
        df['year_week'] = df['fecha'].dt.strftime('%G-W%V')
        training_days_by_week = df.groupby('year_week')['fecha'].nunique().reset_index()
        df = df.merge(training_days_by_week, on='year_week', how='left', suffixes=('','_trained')).drop(columns=['year_week'])
        df.rename(columns={'fecha_trained':'training_days_on_week'}, inplace=True)
    return df

def add_workload_column(df):
    """
    Create a new column with the workload of each set.
    """
    df['workload'] = df.peso * df.reps
    return df

def snake_case(df):
    df.columns = df.columns.str.lower().str.replace(' ','_')

def drop_empty_rows(df, fecha_col: str = 'fecha'):
    df.dropna(axis=0, how='all') # Elinimaos filas donde todos los valores son nulos
    df.drop(df[df[fecha_col].isnull()].index, inplace=True) # Eliminamos filas donde la fecha es nula (just in case)
    return df

def convert_date_columns(df, date_columns: list, multiple_formats=False, desired_format='%Y-%m-%d'):
    """
    Converts specified columns in a DataFrame to datetime format.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        date_columns (list): List of column names to convert to datetime.
        multiple_formats (bool): Indicate if there are multiple columns within the same column.
        desired_format (str): Format used by default.

    Returns:
        pd.DataFrame: DataFrame with converted date columns.
    """
    if multiple_formats == False:
        df[date_columns] = df[date_columns].apply(pd.to_datetime, format=desired_format)
    else:
        for col in date_columns:
            df['fecha_1'] = pd.to_datetime(df[col], format='%Y - %m', errors='coerce')
            df['fecha_2'] = pd.to_datetime(df[col], format='%Y - %m - %d', errors='coerce')
            df['fecha_3'] = pd.to_datetime(df[col], format='%Y-%m-%d', errors='coerce')
            df[col] = df[['fecha_1','fecha_2','fecha_3']].max(1)
            df.drop(['fecha_1', 'fecha_2','fecha_3'], axis=1, inplace=True)

    return df

def convert_right_formats(df, int_cols: list = ['reps','rir'], float_cols: list = ['peso']):
    # Right formats
    df = df.replace('',np.nan)
    df[float_cols] = df[float_cols].astype(float)
    df[int_cols] = df[int_cols].astype(int)
    return df

def basic_cleanings(df: pd.DataFrame, 
                    date_columns: list = ['fecha'], 
                    int_cols: list = ['reps','rir'], 
                    float_cols: list = ['peso']):
    snake_case(df)
    df = drop_empty_rows(df)
    df = convert_date_columns(df, date_columns=date_columns)
    df = convert_right_formats(df, int_cols=int_cols, float_cols=float_cols)
        
    return df

def add_1rm_columns(df):
  # Crear columna con las repeticiones efectivas (reps hechas + RIR)
  df['reps_potential'] = df.repreal + df.rir.replace(-1,0).fillna(0)

  df['1rm'] = np.where((df.real_weight < 50) | (df.reps_potential <= 0), np.nan, # Filtramos ejercicios con pocas cargas
                  np.where(df.repreal <= 8, df.real_weight / (1.0278 - (0.0278 * df.reps_potential)),  # Brzycki
                            np.nan  # Fuera de rango de precisión
                    )
                  ).astype(float).round(1)
  
  # Identify the rows with the max 1RM
  max_rm_exercise = df.loc[df['1rm'].notnull()].groupby('exercise')['1rm'].max().reset_index()
  max_rm_exercise['is_maxrm'] = 1
  df['is_maxrm'] = 0
  df = df.merge(max_rm_exercise, how='left', on=['exercise','1rm'], suffixes=('','_new'))
  df['is_maxrm'] = df[['is_maxrm','is_maxrm_new']].max(1)

  df.drop(['is_maxrm_new'], axis=1, inplace=True)

  return df

def add_category_cols(df):
    """
    Create ranges for reps and rir for further analysis.
    """
    df['repreal_range'] = pd.cut(df.repreal, 
                                 bins=[-1,6,10,15,np.inf], 
                                 labels=['Fuerza','Hipertrofía-Fuerza','Hipertrofía-Resistencia','Resistencia']
                                 )
    df['rir_range'] = pd.cut(df.rir, 
                             bins=[-2,0,3,4,np.inf], 
                             labels=['F|0','1|2|3','4','+5']
                             )

    return df

def define_progression_exercises(df, progression_exercises: dict):
    df['progress_tracker'] = df.exercise.map(progression_exercises)
    return df

In [90]:
df = pd.read_csv(r"..\test_validated_df.csv")

In [91]:
df = basic_cleanings(df)
df.rename(columns={'ejercicio':'exercise','reps':'repreal','peso':'weight'}, inplace=True)

range_sep = df['rango'].str.split(' - ', expand=True).rename(columns={0: 'repmin', 1: 'repmax'})
df = pd.concat([df, range_sep], axis=1).drop(columns=['rango'])
df["technique"] = df["repmin"].where(df["repmin"].str.isalpha())
df['repmin'] = pd.to_numeric(df['repmin'], errors='coerce')
df['workload'] = df.repreal * df.weight
df['effective_set'] = np.where(df.rir <= 4, 1, 0)

df = create_real_weight_column(df=df)
df = training_days_on_week(df)
df = add_1rm_columns(df)
df = add_category_cols(df)

progression_exercises = {
    'Pull-ups': 'Compound', # Tirón vertical
    'Romanian deadlift': 'Compound', # Bisagra de cadera
    'Parallel bar dips': 'Compound', # Empuje Horizontal
    'Smith machine squat': 'Compound', # Sentadilla
    'Preacher curl machine': 'Isolate',
    'Dumbbell lateral raise': 'Isolate',
    'Incline machine press': 'Isolate',
    'Machine row': 'Isolate',
    'Calf raise on machine': 'Isolate'
}

df = define_progression_exercises(df, progression_exercises)

In [92]:
df

,fecha,exercise,repreal,weight,rir,repmin,repmax,technique,workload,effective_set,current_bodyweight,real_weight,training_days_on_week,reps_potential,1rm,is_maxrm,repreal_range,rir_range,progress_tracker
0,2025-09-30,Scott machine bicep curl,5,45.0,2,6.0,8,NaN,225.0,1,NaN,45.0,1,7,NaN,0.0,Fuerza,1|2|3,NaN
1,2025-09-30,Scott machine bicep curl,5,45.0,3,6.0,8,NaN,225.0,1,NaN,45.0,1,8,NaN,0.0,Fuerza,1|2|3,NaN
2,2025-09-30,Scott machine bicep curl,5,45.0,4,6.0,8,NaN,225.0,1,NaN,45.0,1,9,NaN,0.0,Fuerza,4,NaN
3,2025-09-30,Scott machine bicep curl,5,45.0,1,6.0,8,NaN,225.0,1,NaN,45.0,1,6,NaN,0.0,Fuerza,1|2|3,NaN
4,2025-09-30,Incline barbell press,1,135.0,2,78.5,None,NaN,135.0,1,NaN,135.0,1,3,142.9,0.0,Fuerza,1|2|3,NaN
5,2025-09-30,Incline barbell press,2,134.0,2,81.0,None,NaN,268.0,1,NaN,134.0,1,4,146.2,1.0,Fuerza,1|2|3,NaN


In [26]:
workouts = connector.query_to_dataframe("SELECT * FROM workouts")
exercises = connector.query_to_dataframe("SELECT * FROM exercises")

🔍 Conexión detectada
🔍 Conexión detectada


In [8]:
workouts

,id_set,routine,fecha,exercise,repmin,repmax,repreal,weight,rir,workload,technique,real_weight,effective_set,training_days_on_week,1rm,is_maxrm,repreal_range,rir_range,progress_tracker
0,1,Upper I,2023-01-16,Low cable crossover,6.0,12.0,10,10,1,100,None,10,1,4,NaN,0,Hipertrofía-Fuerza,1|2|3,None
1,2,Upper I,2023-01-16,Low cable crossover,6.0,12.0,10,10,3,100,None,10,1,4,NaN,0,Hipertrofía-Fuerza,1|2|3,None
2,3,Upper I,2023-01-16,Low cable crossover,6.0,12.0,10,10,4,100,None,10,1,4,NaN,0,Hipertrofía-Fuerza,4,None
3,4,Upper I,2023-01-16,Chin-ups,1.0,NaN,1,100,1,100,None,110,1,4,113.0,0,Fuerza,1|2|3,None
4,5,Upper I,2023-01-16,Chin-ups,5.0,NaN,5,80,5,400,None,90,0,4,120.0,0,Fuerza,5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10078,10079,Upper II,2025-05-01,Dumbbell lateral raise,-1.0,NaN,8,16,0,128,D,16,1,3,NaN,0,Hipertrofía-Fuerza,F|0,Isolate
10079,10080,Upper II,2025-05-01,Unilateral french press,10.0,15.0,15,14,2,210,None,14,1,3,NaN,0,Hipertrofía-Resistencia,1|2|3,None
10080,10081,Upper II,2025-05-01,Unilateral french press,10.0,15.0,15,14,0,210,None,14,1,3,NaN,0,Hipertrofía-Resistencia,F|0,None
10081,10082,Upper II,2025-05-01,Unilateral french press,10.0,15.0,13,14,0,182,None,14,1,3,NaN,0,Hipertrofía-Resistencia,F|0,None
